# Simple Returns vs Log Returns

The two types act very differently when it comes to aggregation.  Each has an advantage over the other:

    * simple returns aggregate across assets
    * log returns aggregate across time

The simple return of a portfolio is the weighted sum of the simple returns of the constituents of the portfolio.

The log return for a time period is the sum of the log returns of partitions of the time period.  For example the log return for a year is the sum of the log returns of the days within the year.

## 01. Computation

In [ ]:
!pip install yfinance

In [41]:
from datetime import datetime, timedelta
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt 
import yfinance as yf
import seaborn as sns 

sns.set()
pd.options.display.float_format = '{:.4f}'.format 

### 01.01. Simple Return

In [42]:
n_years = 5
def get_return_values(symbol,years):
  end = datetime.now()
  start = end - timedelta(days = n_years * 365)
  price = yf.download(symbol,start,end)
  values  = price['Close'].to_frame()
  values['Returns'] = values['Close'].pct_change()
  return values


In [43]:
values = get_return_values('WMT',n_years)
values.rename({'Close': 'Price'},axis = 1,inplace = True)

[*********************100%***********************]  1 of 1 completed


In [44]:
values.head()

,Price,Returns
Date,,
2017-01-09,68.7100,nan
2017-01-10,68.2300,-0.0070
2017-01-11,68.5300,0.0044
2017-01-12,67.9700,-0.0082
2017-01-13,67.1300,-0.0124


#### Arithmetic Mean Returns

In [45]:
values.Returns.mean()

0.0006828754812254792

#### Geometric Mean / Compound Simple Returns


$ multiple^{\frac{1}{periods}} -1$

In [46]:
multiple = (1 + values.Returns).prod()
multiple

2.0966380363574944

In [47]:
periods = values.Returns.count()

In [48]:
geo_mean = multiple**(1/periods) - 1
geo_mean

0.0005886748917112161

In [49]:
values.Price[-1]

144.05999755859375

In [50]:
values.Price[0] * (1 + geo_mean)**periods

144.0599975586014

####  Normalized Priced (Base 1)

In [51]:
values.Returns.add(1).cumprod()

Date
2017-01-09      nan
2017-01-10   0.9930
2017-01-11   0.9974
2017-01-12   0.9892
2017-01-13   0.9770
              ...  
2021-12-31   2.1058
2022-01-03   2.1052
2022-01-04   2.0667
2022-01-05   2.0946
2022-01-06   2.0966
Name: Returns, Length: 1259, dtype: float64

#### CAGR

In [52]:
multiple ** (1/n_years) - 1

0.159590615735812

### 01.02. Log Returns

In [53]:
values['Log_Returns'] = np.log(values['Price']/values['Price'].shift())

In [54]:
values.head()

,Price,Returns,Log_Returns
Date,,,
2017-01-09,68.7100,nan,nan
2017-01-10,68.2300,-0.0070,-0.0070
2017-01-11,68.5300,0.0044,0.0044
2017-01-12,67.9700,-0.0082,-0.0082
2017-01-13,67.1300,-0.0124,-0.0124


In [55]:
values.describe()

,Price,Returns,Log_Returns
count,1259.0000,1258.0000,1258.0000
mean,110.4903,0.0007,0.0006
std,24.6047,0.0138,0.0137
min,65.6600,-0.1018,-0.1074
25%,88.9600,-0.0055,-0.0055
50%,109.6500,0.0006,0.0006
75%,135.7350,0.0066,0.0066
max,152.7900,0.1171,0.1107


#### Mean Log Return / Reward

In [56]:
values.Log_Returns.mean()

0.0005885016906165329

In [57]:
values.Price[-1]

144.05999755859375

##### Multiple

In [58]:
np.exp(values.Log_Returns.sum())

2.0966380363574917

##### Additive over time

In [60]:
 values.Price[0] *np.exp(values.Log_Returns.sum())

144.0599975585938

In [27]:
values.Price[0] * np.exp(values.Log_Returns.mean() * periods)

143.1799926757814

#### Standard Deviation Log Return/ Risk 

In [13]:
values.Log_Returns.std()

0.01370790532454509

#### Normalized Price (Based 1)

In [61]:
values['Price'] = np.exp(values.Log_Returns.cumsum())
values

,Price,Returns,Log_Returns
Date,,,
2017-01-09,nan,nan,nan
2017-01-10,0.9930,-0.0070,-0.0070
2017-01-11,0.9974,0.0044,0.0044
2017-01-12,0.9892,-0.0082,-0.0082
2017-01-13,0.9770,-0.0124,-0.0124
...,...,...,...
2021-12-31,2.1058,0.0106,0.0106
2022-01-03,2.1052,-0.0003,-0.0003
2022-01-04,2.0667,-0.0183,-0.0185


#### CAGR

In [62]:
trading_days_year = values.Returns.count() / n_years
trading_days_year

251.6

In [63]:
np.exp(trading_days_year * values.Log_Returns.mean()) - 1

0.1595906157358118